In [ ]:
# === 입력 경로 ===
HE_PATH         = "/data/project/banana9903_xenium/xenium_rep1_io/data/xenium/outs/slide_HE_rep1.ome.tif"     # Explorer 때 썼던 H&E 원본
AFFINE_CSV      = "/data/project/banana9903_xenium/xenium_rep1_io/data/matrix.csv"                   # 3x3 homogeneous (Xenium Explorer export)
KEYPOINTS_CSV   = "/data/project/banana9903_xenium/xenium_rep1_io/data/keypoints.csv"                # fixedX,fixedY,alignmentX,alignmentY

# Xenium 좌표 소스 (둘 중 하나 선택)
CELLS_CSV       = "/data/project/banana9903_xenium/xenium_rep1_io/data/xenium/outs/cells.csv.gz"                 # morphology 픽셀 좌표 (권장)
XENIUM_H5AD     = "/data/project/banana9903_xenium/xenium_rep1_io/data/merged_with_celltype_raw.h5ad"  # 예: "/path/to/xenium.h5ad"      # µm 좌표(쓰려면 UM2PX 필요)

# DAPI morphology OME-TIFF (있으면 µm→px 자동 추정에 사용)
DAPI_OME_PATH   = "/data/project/banana9903_xenium/xenium_rep1_io/data/xenium/outs/morphology_mip.ome.tif"       # 없으면 None 가능

# h5ad 좌표를 쓸 때만: µm→px (DAPI OME에서 자동 추출되면 무시됨)
UM2PX_MANUAL    = 1.0/0.2125  # 예: 1.0/0.2125 ≈ 4.705882

# 축 보정 스캔(자동 탐색에 포함되지만, 고정하고 싶으면 여기서 지정)
SWAPXY_DEFAULT  = None  # None이면 탐색, True/False로 고정 가능
FLIPY_DEFAULT   = None  # None이면 탐색, True/False로 고정 가능

# 프리뷰(커다란 이미지일 때 화면용 리사이즈)
PREVIEW_MAX_W   = 6000


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, tifffile as tiff
from itertools import product

def read_he_rgb(path, series_index=0):
    with tiff.TiffFile(path) as tf:
        s = tf.series[series_index]
        a = s.asarray()
    a = np.squeeze(a)
    # (C,H,W) → (H,W,C)
    if a.ndim==3 and a.shape[0] in (3,4) and a.shape[-1] not in (3,4):
        a = np.moveaxis(a, 0, -1)
    if a.ndim==2:
        a = np.stack([a]*3, axis=-1)
    if a.shape[-1]==4:
        a = a[...,:3]
    a = a.astype(np.float32)
    vmin, vmax = np.percentile(a, [1,99]); vmax = max(vmax, vmin+1)
    a = np.clip((a-vmin)/(vmax-vmin), 0,1)
    return (a*255).astype(np.uint8)

def load_affine_csv(p):
    M = np.loadtxt(p, delimiter=",")
    M = np.asarray(M, float)
    if M.shape != (3,3):
        raise ValueError(f"Affine CSV must be 3x3; got {M.shape}")
    return M

def apply_affine(xy, H):
    xy1 = np.c_[xy, np.ones(len(xy))]
    uv1 = xy1 @ H.T
    return uv1[:, :2] / uv1[:, 2:3]

def overlay(img, pts, title="overlay", s=2, edge="cyan"):
    H,W = img.shape[:2]
    plt.figure(figsize=(max(6, W/800), max(6, H/800)))
    ax = plt.gca(); ax.imshow(img, origin="upper")
    ax.scatter(pts[:,0], pts[:,1], s=s, facecolors="none",
               edgecolors=edge, linewidths=0.6, alpha=0.9)
    ax.set_xlim(0, W); ax.set_ylim(H, 0)
    ax.set_title(title); ax.set_xlabel("x [px]"); ax.set_ylabel("y [px]")
    plt.tight_layout(); plt.show()

def inside_ratio(shape, pts):
    H,W = shape[:2]
    m = (pts[:,0]>=0)&(pts[:,0]<W)&(pts[:,1]>=0)&(pts[:,1]<H)
    return float(m.mean())

def get_um2px_from_ome(path):
    if not path: return None
    with tiff.TiffFile(path) as tf:
        omexml = tf.ome_metadata or ""
    import re
    m = re.search(r'PhysicalSizeX="([\d\.eE+-]+)"', omexml)
    if not m: return None
    um_per_px = float(m.group(1))
    return (1.0/um_per_px) if um_per_px>0 else None


In [ ]:
# 3×3 행렬 + 키포인트 로드
H_csv = load_affine_csv(AFFINE_CSV)
kp = pd.read_csv(KEYPOINTS_CSV)
# assume columns order: fixedX,fixedY,alignmentX,alignmentY
fixed     = kp[[kp.columns[2], kp.columns[3]]].to_numpy(float)   # H&E px
alignment = kp[[kp.columns[0], kp.columns[1]]].to_numpy(float)   # Xenium morphology px

# 두 가정 비교
pred1 = apply_affine(alignment, H_csv)   # alignment→fixed 가정
rmse1 = float(np.sqrt(np.mean((pred1 - fixed)**2)))
pred2 = apply_affine(fixed, H_csv)       # fixed→alignment 가정
rmse2 = float(np.sqrt(np.mean((pred2 - alignment)**2)))

print(f"RMSE(alignment→fixed) = {rmse1:.2f} px")
print(f"RMSE(fixed→alignment) = {rmse2:.2f} px")

if rmse1 <= rmse2:
    print("✔ 행렬 방향: alignment→fixed (Xenium morphology px → H&E px)")
    H_coord2img = H_csv
else:
    print("✔ 행렬 방향: fixed→alignment (H&E→Xenium) → 좌표→H&E에는 역행렬 사용")
    H_coord2img = np.linalg.inv(H_csv)

# 키포인트 시각 검증
he = read_he_rgb(HE_PATH)
overlay(he, apply_affine(alignment, H_coord2img),
        title=f"Keypoints projected (RMSE={min(rmse1,rmse2):.1f}px)", s=30)


In [ ]:
if CELLS_CSV is not None:
    dfc = pd.read_csv(CELLS_CSV)
    cols = {c.lower(): c for c in dfc.columns}
    xcol = cols.get("x_centroid") or cols.get("x") or cols.get("px_x")
    ycol = cols.get("y_centroid") or cols.get("y") or cols.get("px_y")
    assert xcol and ycol, f"좌표 컬럼을 찾지 못했습니다: {list(dfc.columns)[:12]}"

    xy_morph_px = dfc[[xcol, ycol]].to_numpy(float)      # morphology px
    xy_on_he = apply_affine(xy_morph_px, H_coord2img)

    ratio = inside_ratio(he.shape, xy_on_he)
    print(f"Inside H&E frame: {ratio*100:.1f}% of points")
    # 프리뷰 리사이즈
    H,W = he.shape[:2]
    if max(H,W) > PREVIEW_MAX_W:
        import cv2
        scale = PREVIEW_MAX_W / max(H,W)
        he_small = cv2.resize(he, (int(W*scale), int(H*scale)), interpolation=cv2.INTER_AREA)
        overlay(he_small, xy_on_he*scale, title=f"Cells→H&E (preview scale={scale:.3f})")
    else:
        overlay(he, xy_on_he, title="Cells (morphology px) → H&E")
else:
    print("CELLS_CSV 경로가 None 입니다. h5ad를 사용하는 다음 셀을 실행하세요.")


In [ ]:
if XENIUM_H5AD is not None:
    import scanpy as sc
    ad = sc.read_h5ad(XENIUM_H5AD)
    assert "spatial" in ad.obsm_keys(), ".obsm['spatial']가 없습니다."
    xy_um = np.asarray(ad.obsm["spatial"], float)[:, :2]

    um2px_auto = get_um2px_from_ome(DAPI_OME_PATH)
    UM2PX = UM2PX_MANUAL or um2px_auto
    if UM2PX is None:
        raise RuntimeError("UM2PX를 알 수 없습니다. DAPI OME-XML에서 PhysicalSizeX를 읽거나 UM2PX_MANUAL 지정 필요.")

    print(f"UM2PX = {UM2PX}  (auto from OME: {um2px_auto})")

    # 조합 탐색 (swap/flip 고정값이 주어지면 그 값만 사용)
    swap_opts = [SWAPXY_DEFAULT] if SWAPXY_DEFAULT is not None else [False, True]
    flip_opts = [FLIPY_DEFAULT]  if FLIPY_DEFAULT  is not None else [False, True]

    best = (-1, None, None, None)
    for swap, flip in product(swap_opts, flip_opts):
        xy = xy_um.copy() * float(UM2PX)
        if swap: xy = xy[:, ::-1]
        if flip: xy[:,1] = -xy[:,1]
        xy_on = apply_affine(xy, H_coord2img)
        r = inside_ratio(he.shape, xy_on)
        print(f"swap={swap}, flip={flip} -> inside={r*100:.1f}%")
        if r > best[0]:
            best = (r, swap, flip, xy_on)

    print("\nBEST:", f"swap={best[1]}", f"flip={best[2]}", f"inside={best[0]*100:.1f}%")
    overlay(he, best[3], title=f"h5ad(µm) → px → H&E (swap={best[1]}, flip={best[2]})")
else:
    print("XENIUM_H5AD 경로가 None 입니다. h5ad 변환 단계는 생략합니다.")


In [ ]:
# 키포인트의 fixed가 현재 HE에 그대로 올라가는지 확인 (해상도/파일 불일치 디버그)
overlay(he, fixed, title="Fixed keypoints on THIS H&E (yellow)", s=40, edge="yellow")


In [ ]:
# --- 설정 ---
PATCH_SIZE = 224
OUT_DIR = "./patches_224"
PAD_MODE = "edge"  # 경계 넘어가면 가장자리 픽셀로 패딩
UM2PX = 4.705882352941177  # 확정
SWAPXY = False
FLIP_Y = False

import os, math, numpy as np, pandas as pd, tifffile as tiff
os.makedirs(OUT_DIR, exist_ok=True)

def apply_affine(xy, H):
    xy1 = np.c_[xy, np.ones(len(xy))]
    uv1 = xy1 @ H.T
    return uv1[:, :2] / uv1[:, 2:3]

def safe_crop_rgb(img, cx, cy, size=224, pad_mode="edge"):
    H, W = img.shape[:2]
    half = size//2
    x0, y0 = int(round(cx))-half, int(round(cy))-half
    x1, y1 = x0+size, y0+size
    ix0, iy0 = max(x0,0), max(y0,0)
    ix1, iy1 = min(x1,W), min(y1,H)
    patch = np.zeros((size,size,3), dtype=img.dtype)
    if pad_mode == "edge":
        sx0, sy0 = min(max(x0,0), W-1), min(max(y0,0), H-1)
        patch[:] = img[sy0, sx0]
    if ix1>ix0 and iy1>iy0:
        px0, py0 = ix0-x0, iy0-y0
        px1, py1 = px0+(ix1-ix0), py0+(iy1-iy0)
        patch[py0:py1, px0:px1] = img[iy0:iy1, ix0:ix1]
    return patch

# 1) 좌표 로드 (cells.csv 기준; h5ad면 xy_um만 바꿔 끼우면 됨)
df = pd.read_csv(CELLS_CSV)
cols = {c.lower(): c for c in df.columns}
xcol = cols.get("x_centroid") or cols.get("x") or cols.get("px_x")
ycol = cols.get("y_centroid") or cols.get("y") or cols.get("px_y")
idcol= cols.get("cell_id") or cols.get("id")
ids  = (df[idcol].astype(str).to_numpy() if idcol else np.array([f"cell{i}" for i in range(len(df))]))
xy_um = df[[xcol, ycol]].to_numpy(float)     # 현재 좌표가 µm임

# 2) µm→px & 축 보정
xy_px = xy_um * UM2PX
if SWAPXY: xy_px = xy_px[:, ::-1]
if FLIP_Y: xy_px[:,1] = -xy_px[:,1]

# 3) morphology px → H&E px
xy_on_he = apply_affine(xy_px, H_coord2img)

# 4) 패치 저장
heH, heW = he.shape[:2]
half = PATCH_SIZE//2
valid = (
    (xy_on_he[:,0]>=-half) & (xy_on_he[:,0]<heW+half) &
    (xy_on_he[:,1]>=-half) & (xy_on_he[:,1]<heH+half)
)
meta = []
for i in np.where(valid)[0]:
    cx, cy = xy_on_he[i]
    patch = safe_crop_rgb(he, cx, cy, size=PATCH_SIZE, pad_mode=PAD_MODE)
    fname = os.path.join(OUT_DIR, f"{ids[i]}_x{int(round(cx))}_y{int(round(cy))}_{PATCH_SIZE}.png")
    tiff.imwrite(fname, patch, photometric='rgb')
    meta.append({"cell_id": ids[i], "x_he_px": float(cx), "y_he_px": float(cy), "patch_path": fname})

pd.DataFrame(meta).to_csv(os.path.join(OUT_DIR, f"patch_meta_{PATCH_SIZE}.csv"), index=False)
print(f"saved {len(meta)} patches → {OUT_DIR}")
print(f"patch physical size ≈ {PATCH_SIZE*0.2125:.1f} µm")


In [ ]:
# === Spot → Patch 매핑 CSV 생성 ===
import os, numpy as np, pandas as pd

heH, heW = he.shape[:2]
half = PATCH_SIZE // 2

# 패치가 이미지 경계를 넘어가도 저장은 했으니, 매핑은 전체를 다 남긴다.
# 다만 H&E 유효 영역 안/밖 여부 플래그를 함께 기록.
inside = (
    (xy_on_he[:,0] >= 0) & (xy_on_he[:,0] < heW) &
    (xy_on_he[:,1] >= 0) & (xy_on_he[:,1] < heH)
)

# (저장할 때 썼던 파일명 규칙과 동일하게) 파일명 구성
def patch_filename(cell_id, cx, cy, size):
    return f"{cell_id}_x{int(round(cx))}_y{int(round(cy))}_{size}.png"

rows = []
for i in range(len(ids)):
    cx, cy = float(xy_on_he[i,0]), float(xy_on_he[i,1])
    x0, y0 = int(round(cx)) - half, int(round(cy)) - half
    x1, y1 = x0 + PATCH_SIZE, y0 + PATCH_SIZE

    fname = patch_filename(ids[i], cx, cy, PATCH_SIZE)
    fpath = os.path.join(OUT_DIR, fname)

    rows.append({
        "cell_id": ids[i],
        # 원본 cells.csv 좌표(µm 가정)
        "x_um": float(xy_um[i,0]),
        "y_um": float(xy_um[i,1]),
        # µm→px 변환 뒤 (morphology 픽셀)
        "x_morph_px": float(xy_px[i,0]),
        "y_morph_px": float(xy_px[i,1]),
        # 최종 H&E 픽셀 좌표(패치 중심)
        "x_he_px": cx,
        "y_he_px": cy,
        # 패치 박스(H&E 픽셀, 좌상단/우하단)
        "x0_he_px": x0, "y0_he_px": y0,
        "x1_he_px": x1, "y1_he_px": y1,
        # 유효영역 플래그
        "inside_he": bool(inside[i]),
        # 패치 파일
        "patch_filename": fname,
        "patch_path": fpath,
        "patch_size_px": PATCH_SIZE
    })

map_df = pd.DataFrame(rows)

# 원한다면 cells.csv의 메타(예: cell_type) 붙이기
# 예: 'cell_type' 컬럼이 있다면 아래 주석 해제
# cols_lower = {c.lower(): c for c in df.columns}
# cid_col = cols_lower.get("cell_id") or cols_lower.get("id")
# if cid_col:
#     map_df = map_df.merge(df[[cid_col, "cell_type"]].rename(columns={cid_col:"cell_id"}),
#                           on="cell_id", how="left")

# 저장
map_csv = os.path.join(OUT_DIR, f"spot_patch_map_{PATCH_SIZE}.csv")
map_df.to_csv(map_csv, index=False)
print(f"[OK] spot→patch 매핑 CSV 저장: {map_csv}")
print(f"총 {len(map_df)}행 | inside_he=True 비율: {map_df['inside_he'].mean()*100:.1f}%")
map_df.head(3)
